## Special Skript for the 20220412 Emergence Experiment

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from evaluation_scripts.experiment_class import Experiment
import os

experiments = ["20220412", "20220127", "20210417"]

## Advanced Phenotyping
- Only take care of easy cases
    - Ar + eff treatment = A^l etc.
    - Therefore only prev_pheno == current_pheno
    - If resulting pheno is 



In [2]:
def check_if_treatment_was_effective_backup(row, df, case):
    effective_treatments = {
        "S":["A", "B", "AB"],
        "A_r":["B", "AB"],
        "B_r":["A", "AB"],
    }  
    
    if case == "transfer":
        i = row.transfer_from_well_id
        prev_phen = df.loc[i,"phenotype"]
    if case == "turnover":
        prev_phen = row.turnover_strain
    
    treat = row["treatment_with"]
    current_phen = row.phenotype
    
    if (prev_phen == current_phen) & (current_phen in effective_treatments.keys()):
        eff_treats = effective_treatments[current_phen]
        if treat in eff_treats:
            return current_phen+"_*"
        else:
            return current_phen+"_tau"
    else: 
        return current_phen+"_?"


def get_advanced_phenotype(row):
    effective_treatments = {
        "S":["A", "B", "AB"],
        "A_r":["B", "AB"],
        "B_r":["A", "AB"],
    }  
    current_phen = row.phenotype
    prev_phen = row.x
    treat = row.treatment_with
    if (prev_phen == current_phen) & (current_phen in effective_treatments.keys()):
        eff_treats = effective_treatments[current_phen]
        if treat in eff_treats:
            return current_phen+"^l"
        else:
            return current_phen+"^h"
    else: 
        return current_phen+"^?"

def added_phenotypes(row, data, advanced= False):
    v = []
    for w in row.added_wells:
        # Turnover
        if "S" in w: 
            advanced_pheno = row.turnover_strain + "^h"
        # Transfer or infection
        elif w in data.index:
            if data.loc[w, "exclude"]:
                advanced_pheno = "?"
            else:
                advanced_pheno = data.loc[w, "advanced_phenotype"]
        else:
            advanced_pheno = "?"
        if (advanced_pheno not in v) & (type(advanced_pheno) == str):
            v.append(advanced_pheno)
    return v

In [3]:
for exp in experiments:
    print(exp)
    experiment = Experiment(experiments[0])
    data = experiment.data
    data["advanced_phenotype"] = data.apply(lambda x: get_advanced_phenotype(x), axis = 1)
    data["added_advanced_phenotypes"] = data.apply(lambda x: added_phenotypes(x, data, advanced = True), axis=1)

    ## Save
    data.to_pickle(os.path.join(experiment.pathes["obj"],"02g_advanced_phenotyping.pkl"))

20220412
20220127
20210417


In [4]:
data[data.x == "A&B"].added_advanced_phenotypes.head(50)

ID
t1_P1_A3     [B_r^h, A_r^h]
t1_P1_A4     [B_r^h, A_r^h]
t1_P1_B3     [B_r^h, A_r^h]
t1_P1_B4     [B_r^h, A_r^h]
t1_P1_A5     [B_r^h, A_r^h]
t1_P1_A6     [B_r^h, A_r^h]
t1_P1_B5     [B_r^h, A_r^h]
t1_P1_B6     [B_r^h, A_r^h]
t1_P1_A7     [A_r^h, B_r^h]
t1_P1_A8     [A_r^h, B_r^h]
t1_P1_B7     [A_r^h, B_r^h]
t1_P1_B8     [A_r^h, B_r^h]
t1_P1_A21    [A_r^h, A&B^?]
t1_P1_A22    [A_r^h, B_r^h]
t1_P1_B21    [A_r^h, B_r^h]
t1_P1_B22    [A_r^h, B_r^h]
t1_P1_C11    [B_r^h, A&B^?]
t1_P1_C13    [B_r^h, A_r^h]
t1_P1_C14    [B_r^h, A_r^h]
t1_P1_D13    [B_r^h, A_r^h]
t1_P1_D14    [B_r^h, A_r^h]
t1_P1_E11    [A_r^h, B_r^h]
t1_P1_E12    [A_r^h, B_r^h]
t1_P1_F11    [A_r^h, B_r^h]
t1_P1_F12    [A_r^h, B_r^h]
t1_P1_H8            [A&B^?]
t1_P1_G11           [A&B^?]
t1_P1_G19    [A_r^h, B_r^h]
t1_P1_G20    [A_r^h, B_r^h]
t1_P1_H19    [A_r^h, B_r^h]
t1_P1_H20    [A_r^h, B_r^h]
t1_P1_I9            [A&B^?]
t1_P1_I21           [A&B^?]
t1_P1_J22           [A&B^?]
t1_P1_L4            [A&B^?]
t1_P1_K15    [B_r